In [5]:
%cd ~/change
import pickle
import torch
import random
from transformers import AutoTokenizer,AutoModelForMaskedLM,Trainer,TrainingArguments
with open('data/mopen.pickle',mode='rb') as f:
    train_x = pickle.load(f)
    train_y = pickle.load(f)

/home/sibava/change


In [6]:
tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-v2')
model = AutoModelForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-v2')

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
texts = []
labels = []
def select_y(y):
    s = random.randrange(0,2)
    if(y == 'からが'):
        if(s == 0):
            y = 'から'
        else:
            y = 'が'
    elif(y == 'にを'):
        if(s==0):
            y = 'に'
        else:
            y = 'を'
    elif(y == 'がで'):
        if(s==0):
            y = 'が'
        else:
            y = 'で'
    elif(y == 'がに'):
        if(s==0):
            y = 'に'
        else:
            y = 'が'
    elif(y == 'にで'):
        if(s==0):
            y = 'に'
        else:
            y = 'で'
    else:
        y = y
    return y

In [8]:
for x,y in zip(train_x,train_y):
    y = select_y(y)
    texts.append(f'{x[0]}[SEP]{x[1]}[MASK]{x[2]}')
    labels.append(f'{x[0]}[SEP]{x[1]}{y}{x[2]}')
train_texts = tokenizer(texts,truncation=True,padding=True,return_tensors='pt')
train_labels = tokenizer(labels,truncation=True,padding=True,return_tensors='pt').input_ids
class KakuDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val)[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
        # item = {'text':tokenizer.decode(self.encodings[idx].input_ids)}
        # item['labels'] = tokenizer.decode(self.labels[idx].input_ids) 
        # return item
        
    def __len__(self):
        return len(self.labels)
train_dataset = KakuDataset(train_texts,train_labels)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [9]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
)

trainer.train()

***** Running training *****
  Num examples = 1788
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 336
/tmp/ipykernel_2792922/1149289476.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val)[idx] for key, val in self.encodings.items()}
/tmp/ipykernel_2792922/1149289476.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
/home/sibava/miniconda3/envs/kakukoutai/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimens

Step,Training Loss
10,3.368400
20,0.062800
30,0.012900
40,0.007900
50,0.004400
60,0.003000
70,0.009900
80,0.007000
90,0.001600
100,0.001500




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=336, training_loss=0.10483875216041702, metrics={'train_runtime': 212.3394, 'train_samples_per_second': 25.261, 'train_steps_per_second': 1.582, 'total_flos': 372269784176640.0, 'train_loss': 0.10483875216041702, 'epoch': 3.0})

In [10]:
torch.save(model.state_dict(),'models/select_trained2.pth')